# Exercise session 3: Query-based systems

In this session, you will be given access to a private dataset through queries. You will be confronted to different noise addition mechanisms that attempt to preserve the privacy of people in the datasets, and will develop attacks to obtain private information on users. 

## The dataset

This dataset containst records for 32561 people from a census. The columns of this dataset are:
`age`, `workclass`, `fnlwgt`, `education`, `education-num`, `marital-status`, `occupation`, `relationship`, `race`, `sex`, `capital-gain`, `capital-loss`, `hours-per-week`, `native-country`, `salaryclass`. All of the columns are categorical, with values replaced by integers.

This dataset is hosted on a server (whose IP will be given to you). The different data protection mechanisms are hosted available on two different ports. The queries that you are allowed to send belong to a subset of SQL: 

$$\texttt{condition} \ \ \text{and} \ \ \ldots \ \ \text{and} \ \ \texttt{condition}$$

where `condition := columnname '<'|'>'|'<>'|'='|'<='|'>=' value`, and `value` is either an integer or an arithmetic expression that evaluates to an integer (e.g. `(2*2)`). Each of these queries returns the _count_ of users who satisfy all conditions in the query. 

In this notebook, you will try and find the `salaryclass` of some users,  using only queries. For simplicity, `salaryclass` is a binary attribute, where 0 corresponds to standard income and 1 is higher income.

In [1]:
import socket
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def remote_query(query, host, port):
    '''This function uses a very simple socket protocol to send 
       queries to the database system.'''
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((host, port))
        s.sendall(query.encode('utf-8'))
        response = s.recv(1024).decode('utf-8')
        if response.startswith('ERROR '):
            raise Exception(response[6:])
    return eval(response)

The user you are asked to "attack" (let's call this user Alex), has the following attributes:

In [3]:
# Here are the attributes in Python form ;-)

Alex = { 'age':            39,
         'workclass':       7,
         'education':       9,
         'education_num':  13,
         'marital_status':  4,
         'occupation':      1,
         'relationship':    1,
         'race':            4,
         'sex':             1,
         'capital_gain': 2174,
         'capital_loss':    0,
         'hours_per_week': 40,
         'native_country': 39 }

These attributes will constitute your _background knowledge_ on the victim.

## Exercise 1: Two simple mechanisms

In this exercise, you will break two simple mechanisms: noise addition, and query set size restriction. Each mechanism requires a different attack. Then, we will combine these mechanisms, and show that (at least in this case) they do no effectively protect the sensitive information.

All of these queries will be sent to the same server, whose IP will be provided by the TA. The first character of each query will specify which mechanism to use (but we have already taken care of this by defining convenient functions for you to use).

In [4]:
HOST  = "146.169.41.52"
PORT1 = 42420

### Exercise 1a | Noise addition

Firstly, you will interact with a very simple privacy mechanism, noise addition: the query-based system adds a noise $N \sim \mathcal{N}(0, 5)$ to the query's response. This noise is independent for each query.

The following code defines the `query_1a` function that queries this specific system.

In [5]:
# you can use a direct call to `query_1a("your query")` for exercise 1.a
query_1a = lambda query: remote_query('a'+query, HOST, PORT1)

**Step 1:** Execute the following cell several times to see how the mechanism works. Feel free to change the query.

In [6]:
query_1a('age = 32 and sex = 1')

571.1479990679458

Since all you have access to are `COUNT` queries, you can't directly query the system to ask for Alex's `salaryclass`. There is a workaround, however: you can count the number of users who are Alex, _and_ have `salaryclass = 1`.

**Step 2 _(action required)_**: Write and perform a query that targets specifically Alex and tests for `salaryclass = 1`. Is the result informative?

_Your code should define a `query` variable._

In [7]:
# YOUR CODE HERE
query = '' # YOUR QUERY HERE

# ANSWER
query = ' and '.join('%s = %s' % x for x in Alex.items())
query += ' and salaryclass = 1'

# WHAT HAPPENS WHEN YOU QUERY IT?
print( query_1a(query) )

-0.25130386496619983


As you learned in the class, such a simple mechanism falls to simple _averaging attacks_. By the central limit theorem, repeating the query several times and taking the average reduces the variance of the noise, thus converging to the true answer.

**Step 3 _(action required)_**: Given the distribution of the noise, propose a simple criterion to determine Alex's value, given the noisy samples. Compute how many queries you would need to have an error probability of less than 5%. This is a paper-and-pencil exercise.

_Your answer here._

`SAMPLE ANSWER`

Let's start with some notations. Let $b \in \{0,1\}$ the true value for Alex. Each sample is thus $X_i = b + N_i \sim \mathcal{N}(b,5)$. The test we propose to use, if given $n$ results $x_i$, is to estimate Alex's value as $1$ if $\frac{1}{n} \sum_{i=1}^n x_i \geq \frac{1}{2}$ (one can show this is the maximum likelihood estimator). Denote by $\hat{B}$ this estimate. We also denote $Y = \frac{1}{n} \sum_{i=1}^n N_i \sim \mathcal{N}(0, \frac{5}{n})$, the sum of all noise. This implies that $\hat{B}$ is 1 if $b + Y \geq 1/2$ and is 0 otherwise.

There are two possible errors: predicting 1 for a 0, and vice versa: 
1. $P[\hat{B} = 1~|~b=0] = P[Y \geq 1/2]$
2. $P[\hat{B} = 0~|~b=1] = P[1 + Y \leq 1/2] = P[Y \leq -1/2]$

$Y$ is normally distributed, so by symmetry of the Gaussian these two errors are the same.

We define "error" to mean that the prediction we make is different from the true value $b$. This means:
$$P[error] = P\left[\hat{B} = 0, b = 1\right] + P\left[\hat{B} = 1, b = 0\right]$$

This means that the probability of error can be written as follows (independently of the probability distribution of $b$):
$$P[error] = P[\hat{B} = 1~|~b=0] \cdot P[b=0] + P[\hat{B} = 0~|~b=1] \cdot P[b=1] = P[Y \geq 1/2] \cdot (P[b=0] + P[b=1]) = P[Y \geq 1/2]$$

Finally, we impose that this probability is less than 5% (denote $Z \sim \mathcal{N}(0,1)$ the standard normal distribution):
$$P[Y \geq 1/2] = P[Z \geq \frac{1}{2 \sqrt{5/n}}] \leq 0.05$$

Using the CDF of the Gaussian, we find the following contraint on $n$:
$$\frac{\sqrt{n}}{2 \sqrt{5}} \geq z_{0.95} = 1.96,$$
where $z_{0.95}$ is the 95th percentile of the standard normal distribution.

And thus $n \geq 4 \cdot 1.96^2 \cdot 5 = 76.83$. Hence, **77 queries are enough**.

**Step 4 _(action required)_**: Perform the attack, using the `query` defined in step 2 and the number of queries you found in step 3.

In [8]:
# YOUR CODE HERE

# ANSWER
samples = []
for x in range( 77 ):
    samples.append(query_1a(query))

average = np.mean(samples)
decision = average >= 0.5

print('Samples collected:', samples)
print('Average:', average)
print("Alex's value is likely to be", int(decision))

Samples collected: [1.0983957750228894, 0.12292832674495027, 0.30094501737079765, 0.16023611715339606, 0.1915730954643442, -0.8020332051894687, -0.10296488321322804, 2.726690897202801, -2.0748585510426882, 5.9450855318334055, -0.08345717062565695, 4.737621393113159, 0.5315034817046967, 3.192688444608473, 0.41142693734485486, -0.20447369182891575, 3.3986963933847334, 1.388295252802671, 0.2111255923097043, 2.092569791360849, 3.1725184130917943, -0.08058647524363916, -0.29738957643592306, 2.5736208045214886, 2.8295639774326884, 2.9991080602437727, 0.8398752751968995, -2.508327018117927, -2.84100139904295, 2.925034699065157, -2.464422825479331, -0.4885732249281777, 0.7422833766413038, -2.1420737763076767, -1.0781916851142967, 0.6871478265196274, -3.0787370900522983, 4.014596710448741, 3.008624874910219, -0.1420297857984458, -0.7884378911259651, -1.341354226588387, -4.813206809985287, -3.501262230777639, 1.5703651029503654, -1.8484617871196671, 0.2000484918465827, 0.17877382122127233, -1.84

### Exercise 1b | Query set size restriction

The previous mechanism fails easily, because it is easy to target Alex directly. Another option is to _suppress every query that selects too few users_. So we implemented another privacy mechanism that enforces a query set size restriction (without noise addition). Specifically, if the user count in the response is less than **5**, the query returns __-1__.

In [9]:
# as before, you can use a direct call to `query_1b("your query")` for exercise 1.b
query_1b = lambda query: remote_query('b'+query, HOST, PORT1)

**Step 1**: what happens now, when you try your query from exercise 1.a ?

In [10]:
query_1b( query )

-1

In this exercise, you will be asked to attack Bob, whose characteristics are as follows:

In [11]:
# Here are the attributes in Python form ;-)

Bob = { 'age':            50,
        'workclass':       6,
        'education':       9,
        'education_num':  13,
        'marital_status':  2,
        'occupation':      4,
        'relationship':    0,
        'race':            4,
        'sex':             1,
        'capital_gain':    0,
        'capital_loss':    0,
        'hours_per_week': 13,
        'native_country': 39 }

It doesn't work anymore. However, it does not mean that query set size restriction is the solution for privacy. Indeed, this technique is vulnerable to _intersection attacks_. The idea of an intersection attack is to perform two queries whose answers select several users, but with only one user as the difference between them. Thus, the difference of the two queries gives the exact answer for that user.

Typically, they take the form:
1. `Query1 = (condition1 and condition2 and ...)`
2. `Query2 = Query1 and Discriminative condition`

Where `Query1` relates to many users, and `Query2` relates to the same users as `Query1` **except for the target user** (for whom the `Discriminative condition` is false). To find these queries, use the target's values, and assert that the count for `Query2` is the same as for `Query1` _minus 1_.

**Step 2  _(action required)_**: Using queries to the server, find (by trial and error) a pair `(Query1, Query2)` using the data _you know_ about Bob.

_Your code should define two variables, called `query1` and `query2`._

In [12]:
### YOUR CODE HERE
query1 = '' # YOUR QUERY HERE
query2 = '' # YOUR QUERY HERE

### ANSWER
query1 = 'age = 50 and education = 9'
query2 = query1 + ' and hours_per_week <> 13'

# print the content
print(query_1b(query1), query_1b(query2))

# we get: 103 and 102 --> Bob is the only user with (age=50, education=9, hours_per_week=13) and many users
#  have age=50 and education=9

# The idea here is that we used hours_per_week, a high entropy attribute, to discriminate Bob.

103 102


**Step 3 _(action required)_**: Using these queries, find Bob's (exact) secret value (you will need to adapt these queries to take into account the `salaryclass`).

In [13]:
### YOUR CODE HERE

### ANSWER
guess = ' and salaryclass = 1'
difference = query_1b(query1+guess) - query_1b(query2+guess)
print('my guess is: salaryclass = %d' % (difference)) # if query1 = query2, then Bob is not part of query1 or query2 --> Bob's salaryclass is 0

my guess is: salaryclass = 0


### Exercise 1c | Noise addition + Query set size restriction

Finally, we could combine these two mechanisms to have a stronger mechanism. Mechanism 1c returns either (-1) if the query concerns less than 5 users, and otherwise the true answer + independent noise according to $\mathcal{N}(0,5)$.

In [14]:
# you can use a direct call to `query_1c(your_query)` for exercise 1.
query_1c = lambda query: remote_query('c'+query, HOST, PORT1)

**Step 1**: Use your query from exercise 1a. Is the result informative?

In [15]:
query_1c(query)

-1

**Step 2**: Use `query1` and `query2` from exercise 1b. Are the results informative?

In [16]:
print(query_1c(query1), query_1c(query2))

103.16357369239556 106.03602869953296


**Step 3  _(action required)_**: Adapt your attack from exercise 1b to attack this mechanism, and (again) find Bob's secret value. Does combining the mechanisms make them stronger?

In [17]:
### YOUR CODE HERE

### ANSWER
samples = [query_1b(query1+guess) - query_1b(query2+guess) for _ in range(154)]
print('my guess is: salaryclass = %d' % (np.mean(samples) > 0.5))
# Note: we need twice as many queries because the variance of a difference of gaussians is equal to the sum of variances.

my guess is: salaryclass = 0


## Exercise 2

In this second exercise, the query-based system uses _static sticky noise_: it adds noise that depends on the conditions. That is, if a query $Q \equiv$ `C1 and C2 and C3` is issued, our mechanism adds one fixed noise value per condition. The output to Q is then:
$\newcommand{\static}{\operatorname{static}}$

$$\widetilde{Q}(D) = Q(D) + \static[\text{C1}] + \static[\text{C2}] + \static[\text{C3}],$$

where each $\static[\text{C}x]$ is a noise value drawn from a _seeded_ normal distribution. In this exercise, the mechanism draws from $\mathcal{N}(0,2)$. This means that the noise level for one condition is low, but if you have many conditions in your query, the overall noise will be larger than for the first mechanism. The mechanism is seeded such that if a same condition is used several times (say, in subsequent queries), the same noise will be used every time for that condition.

That is, when you query condition `C1`, a noise `n1` is added to the result (always the same noise), and so on. For instance, you would have the following behaviour, if you do these queries (in whatever order):
- `C1` ==> answer1 + `n1`
- `C1 and C2` ==>  answer2 + `n1` + `n2`
- `C1 and C3` ==> answer3 + `n1` + `n3`
- `C1` (again) ==> answer1 + `n1`

In practice, this is implemented by [seeding](https://en.wikipedia.org/wiki/Random_seed) the random number generator (RNG) with a hash of the condition (XORed with some salt). The details of the implementation are not important to solve the exercise, but for more details you can look at the slides from last week.

This mechanism thwarts the attack you developed in exercise 1a: repeating the query will always yield the same result.
To make the mechanism more robust, it also implements query set size restriction, and will return -1 for every query whose user set contains less than 5 users.

In [18]:
# this is running on a different port of the server!
PORT2 = 42422

In [19]:
# same as for exercise 1, the function `query_2` is defined for simplicity of use.
query_2 = lambda query: remote_query(query, HOST, PORT2)

In this exercise, you will attack Carl, whose characteristics are given below.

In [20]:
Carl = {'age':            43,
        'workclass':       4,
        'education':      15,
        'education_num':  10,
        'marital_status':  2,
        'occupation':     13,
        'relationship':    0,
        'race':            2,
        'sex':             1,
        'capital_gain':    0,
        'capital_loss':    0,
        'hours_per_week': 40,
        'native_country': 23 }

**Background knowledge**: from your expert knowledge of the situation, you know that Carl is uniquely identified by his `age`, `marital_status`, and `native_country`. Furthermore, you suspect that _many_ people in the dataset share the same `age` and `marital_status` (hence, the `native_country` seems to be a good discriminative condition).

_Why do we have to assume that the adversary knows that?_

**Step 1**: Perform the following queries. Observe that the result you obtain is the same as your neighbour's. Why is that important?

In [21]:
### try a few queries here
print('Users aged 42:  \t', query_2('age = 42'))
print('Users aged 1000:\t', query_2('age = 1000'))
print('Number of men:  \t', query_2('sex = 0'))

Users aged 42:  	 779.0348179724501
Users aged 1000:	 -1
Number of men:  	 10769.92079293638


**Step 2  _(action required)_**: Perform your attack 1c here (on Carl). Do you think the result is reliable?

In [22]:
## YOUR CODE HERE

## ANSWER
c1 = ['age', 'marital_status']
discr = 'native_country'

query1 = ' and '.join( '%s = %s' % (c, Carl[c]) for c in c1)
query2 = query1 + ' and %s <> %s' % (discr, Carl[discr])

samples = [query_2(query1+guess) - query_2(query2+guess) for _ in range(9)]
print(samples)
print('my guess is: salaryclass = %d' % (np.mean(samples) > 0.5)) # ow no it's wrong (in this particular case)

[-1.6719501719911705, -1.6719501719911705, -1.6719501719911705, -1.6719501719911705, -1.6719501719911705, -1.6719501719911705, -1.6719501719911705, -1.6719501719911705, -1.6719501719911705]
my guess is: salaryclass = 0


_Spoiler alert_ : this mechanism is still not secure. As you see, this mechanism is still vulnerable to **intersection attacks**. Even if the repeated queries return the same result, the noise on the response is actually _very low_. Why is that?

**Step 3  _(action required)_**: What is the total noise on `query_2(query1) - query_2(query2)` ? Why is it low?

_Your answer here._

`SAMPLE ANSWER`

Because `query1` and `query2` differ by only one condition, the difference of the results is thus only the noise corresponding to the `Discriminative condition`. This noise is distributed according to $\mathcal{N}(0,2)$.

While the resulting noise is indeed _small_ , the confidence on your result is low if you can only perform one query. Finding many `(query1, query2)` pairs can be difficult, especially using this more secure interface: it is hard to make sure that Carl is the only user in one query but not the other when the count you find is noisy. In this case, we assume that the pair you already have `(query1, query2)` is _background knowledge_ : something you know about the dataset that you could not figure out by using queries. 

However, there is a trick to get more queries, using this single piece of background knowledge: modifying the queries so that they are _semantically equivalent_ but _syntactically different_. For instance, `sex=0` is equivalent to `sex <> 1`. This is called a **semantic attack**.

**Step 4  _(action required)_**: How many queries do you need for a 95% confidence, given that the noise on one condition is $\mathcal{N}(0,2)$ ?

_Your answer here._

`SAMPLE ANSWER`

The difference between two queries is distributed according to $\mathcal{N}(0,2)$. The development of exercise 1a still holds. Thus $n \geq 4 \cdot 1.96^2 \cdot 2 = 30.7$. Hence, **31 queries are enough**.

**Step 5  _(action required)_**: Vary _syntactically_ the `Discriminative condition` (while keeping the same _semantic_ meaning) to obtain independent samples in the difference attack (by obtaining different expressions for `query2`), in order to have enough queries for a confidence of 95%. Perform the attack. Do you obtain the correct result?

_Hint: this SQL database supports arithmetic operations._

In [23]:
### YOUR CODE HERE

### ANSWER

# we need to find 6 queries ==> 5 equivalents to query2
query2s = [query2]
for x in range(6):
    query2_tmp = query1 + ' and native_country <> (%d + %d)' % (Carl['native_country'] - x, x)
    query2s.append(query2_tmp)

# using these equivalent samples, we can perform the difference attack
samples = [(query_2(query1 + guess) - query_2(q + guess)) for q in query2s]
print(samples)
print('my guess is: salaryclass = %d' % (np.mean(samples) > 0.5)) # yes, it works!

[-1.6719501719911705, -0.4777658357735959, 0.6849249293440494, -0.12391729509866423, -1.5823442389761908, 2.6238365206461367, 2.5388588378699524]
my guess is: salaryclass = 0


In [24]:
query1

'age = 43 and marital_status = 2'

In [25]:
query2

'age = 43 and marital_status = 2 and native_country <> 23'

### Bonus exercise: find the age of a record without background knowledge on uniqueness

In this last part, you will attack the same mechanism as for question 2, but _without information on the attributes to use_. That is, in the last part, we told you that the target was unique according to some attributes, but many other users shared all but one of these attributes. Here, you will have to figure out how to get these attributes.

Furthermore, to make it a bit harder, this time we ask you to obtain the `age` (and optionally `salaryclass`) of this record. As always, you are only allowed to query the system through the limited syntax used in this class. Let us know when you have the age (so we can confirm your result)!

In [26]:
Deborah = {'workclass':       4,
           'education':      15,
           'education_num':  10,
           'marital_status':  0,
           'occupation':      3,
           'relationship':    4,
           'race':            4,
           'sex':             0,
           'capital_gain':    0,
           'capital_loss':    0,
           'hours_per_week': 20,
           'native_country': 39 }

_Hint:_

_**STEP 1:** Find a good subset. You may use subset exploration: iterating over subsets of known attributes until finding a "good" pair (X, y), with X a subset of attributes that very likely contains many users, while y is an attribute that excludes only Deborah from X. "Good" subsets will be identified using a pair of heuristics:_

_1) count(X) is large (not bucket-suppressed)._
_2) count(X and y) is bucket-suppressed._

_NOTE: this is a very naive way of doing this. You are free to explore other possibilities (for instance, using the distribution of the attributes to treat some entries as "rarer"). Note that this will perform very many queries._

_**STEP 2:** From (X,y), compute the age through a difference query (by unnoising the result). In order to do this you may simply modify syntactically the entries by replacing the value of y by (y+1)-1, (y+2)-2, ... and averaging the result._